In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt

from src.core import Domain1D, Domain2D
from src.solvers import Wave, Heat
from src.components import Listener, RickerSource
from src.visualization import PhysicsAnimator
import src.utils as utils

### Comparison Of Impulse Response Estimation from pyroomacoustics

In [ ]:
import pyroomacoustics as pra


# --- CONFIGURATION ---
ROOM_DIM = [10.0, 10.0]
SOURCE_POS = [2.0, 2.0]
MIC_POS = [7.0, 7.0]
R_COEFF = 0.8  # Wall reflection coefficient (Amplitude)
C = 343.0      # Speed of sound

# Calculate derived parameters
alpha_energy = 1.0 - R_COEFF**2
alpha_robin = (1.0 - R_COEFF) / (1.0 + R_COEFF) * (1.0 / C)

print(f"Comparing Simulators with R={R_COEFF}")

# ==========================================
# 1. YOUR FDTD SOLVER
# ==========================================
print("Running FDTD...")
dx = 0.05
domain = Domain2D(length=ROOM_DIM, dx=dx)
solver = Wave(
    domain, 
    initial_u=lambda *args: np.zeros_like(args[0]),
    initial_ut=lambda *args: np.zeros_like(args[0]),
    c=C, 
    boundary_type='robin', 
    alpha=alpha_robin
)

src = RickerSource(pos = SOURCE_POS, peak_freq=200, delay = (1.5 / 200), amplitude=10.0)
solver.add_dynamic_source(src)

mic = Listener(pos=MIC_POS)
solver.add_listener(mic)

# Run for 0.5 seconds
duration = 0.5
steps = int(duration / solver.dt)
for _ in range(steps):
    solver.step()

t_fdtd, sig_fdtd = mic.get_time_series()
# ==========================================
# 2. PYROOMACOUSTICS (Image Source Method)
# ==========================================
print("Running PyRoomAcoustics...")

# 1. Cast fs to int (CRITICAL FIX)
# PRA requires an integer sampling rate. 
# 1/dt is a float (e.g. 4899.75), which causes the C++ TypeError.
pra_fs = int(1 / solver.dt)

# 2. Create room
# Use 'absorption' instead of 'materials' for uniform walls to avoid scalar/array mismatches.
# Lower max_order to ~12 (usually sufficient for T60) unless you need very late reflections.
room = pra.ShoeBox(
    ROOM_DIM, 
    fs=pra_fs, 
    absorption=alpha_energy, 
    max_order=12 
)

# Add source and mic
room.add_source(SOURCE_POS)
room.add_microphone(MIC_POS)

# Compute RIR
room.compute_rir()
sig_pra = room.rir[0][0]
t_pra = np.arange(len(sig_pra)) / room.fs

# ==========================================
# 3. PLOT COMPARISON
# ==========================================
plt.figure(figsize=(12, 6))

# Find the index of the maximum value (Direct Sound)
idx_fdtd = utils.find_first_arrival(sig_fdtd, threshold_ratio=0.15)
idx_pra = utils.find_first_arrival(sig_pra, threshold_ratio=0.15)

# Calculate the time of those peaks
t_peak_fdtd = t_fdtd[idx_fdtd]
t_peak_pra = t_pra[idx_pra]

# Shift FDTD so its peak matches PRA's peak
time_shift = t_peak_fdtd - t_peak_pra
t_fdtd_aligned = t_fdtd - time_shift

norm_fdtd = sig_fdtd / np.max(np.abs(sig_fdtd))
norm_pra = sig_pra / np.max(np.abs(sig_pra))

plt.plot(t_fdtd_aligned, norm_fdtd, label='FDTD (Auto-Aligned)', alpha = 0.6)
plt.plot(t_pra, norm_pra, label='PRA', alpha = 0.6)

plt.title("Comparison: Wave Solver vs. Image Source Method")
plt.xlabel("Time (s)")
plt.ylabel("Normalized Amplitude")
plt.xlim(0, 0.2) # Zoom in on early reflections
plt.legend()
plt.grid(True)
plt.show()

# ==========================================
# 4. MEASURE RT60 ON BOTH
# ==========================================
try:
    rt60_fdtd = pra.experimental.rt60.measure_rt60(sig_fdtd, fs=1/solver.dt, decay_db=30)
    rt60_pra = pra.experimental.rt60.measure_rt60(sig_pra, fs=room.fs, decay_db=30)
    print(f"RT60 FDTD: {rt60_fdtd:.3f} s")
    print(f"RT60 PRA:  {rt60_pra:.3f} s")
except:
    print("Could not calculate RT60 (signals might be too short)")

In [ ]:
solver.reset()
animator = PhysicsAnimator(solver = solver, total_time=0.5)
animator.run()

fig = animator.create_animation(skip_frames=5)
fig.show()

In [ ]:
freqs, fft = mic.compute_spectrum()

In [ ]:
plt.plot(freqs, fft)
plt.xlim([0,1000])